In [1]:
%load_ext autoreload
%autoreload 2

import re, time, json,os,csv
from app.utils import Helper
from datetime import datetime
import pandas as pd
from app.constants import *
from collections import defaultdict

path = INPUT_PATH
def sorted_rate_files(folder_path):
    files = os.listdir(folder_path)
    rate_files = [f for f in files if f.startswith("RealTime_")]
    rate_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))
    return rate_files
 
# Ticker	Date	Time	LTP	BuyPrice	BuyQty	SellPrice	SellQty	LTQ	OpenInterest
# RIIL.NSE	19-05-2025	09:15:03	928.6	0	0	0	0	0	0
# RIIL.NSE	19-05-2025	09:15:03	929.7	926.2	14	929.7	7	1001	0
# RIIL.NSE	19-05-2025	09:15:03	927.95	926.2	14	929.7	7	0	0
# RIIL.NSE	19-05-2025	09:15:15	933.65	930.3	10	933.65	75	2079	0
# RIIL.NSE	19-05-2025	09:15:15	933.65	930.3	10	933.65	75	0	0
# RIIL.NSE	19-05-2025	09:15:25	934	931.45	10	934.45	5	322	0
# RIIL.NSE	19-05-2025	09:15:25	934	931.45	10	934.45	5	0	0
# RIIL.NSE	19-05-2025	09:15:34	931.35	932.2	10	933.95	1	1	0
# RIIL.NSE	19-05-2025	09:15:42	932.4	931.75	10	933.4	3	33	0
# RIIL.NSE	19-05-2025	09:15:51	931.95	931.65	14	933.5	5	275	0

In [3]:
path = r"exchange_mapping.json"
import json
data = Helper.load_json(path)

def_dict = {}

for element in data:
    def_dict.update({element["cogencis_symbol"]:element["exchange_symbol"]})
    

with open("fallback.json","w") as f:
    json.dump(def_dict,f)

In [1]:
from app.ftp_connector import ftp_file_transfer
from app.logger import setup_logger, set_logger
import logging
# Initialize scheduler logger
log_dir = r"C:\Users\kaustubh.keny\Projects\OUTPUTS\ticker-ops\logs"
logger = setup_logger(name="scheduler", log_dir=log_dir, console_level=logging.INFO)
set_logger(logger)
path = r"C:\Users\kaustubh.keny\Downloads\lookup.csv"

ftp_file_transfer(path)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
Traceback (most recent call last):
  File "c:\Users\kaustubh.keny\Projects\office-work\stock-ticker-feed\app\ftp_connector.py", line 28, in ftp_file_transfer
    with FTP(host) as ftp:
         ~~~^^^^^^
  File "c:\Users\kaustubh.keny\AppData\Local\Programs\Python\Python313\Lib\ftplib.py", line 121, in __init__
    self.connect(host)
    ~~~~~~~~~~~~^^^^^^
  File "c:\Users\kaustubh.keny\AppData\Local\Programs\Python\Python313\Lib\ftplib.py", line 158, in connect
    self.sock = socket.create_connection((self.host, self.port), self.timeout,
                ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                                         source_address=self.source_address)
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

In [3]:
path = r"C:\Users\kaustubh.keny\Downloads\SGDataOut17-9-2025.txt"

extensions = set()

with open(path, "r", encoding="utf-8") as f:
    data = f.readlines()
    # for line in f:
    #     print(line.strip())
    for tick in data:
        data = tick.split(";",1)[0]
        data = data.split("CPR")[-1]
        data = data.split(".")[-1]
        
        extensions.add(data)
        

extensions

{'NSF', 'NSFOA', 'NSOG'}

In [4]:
input_folder_path = r"C:\Users\kaustubh.keny\Projects\INPUTS\17-09-2025"

source_total_lines = 0
source_malformed_lines = []
source_symbol_tick_dict = {}


# indexes = ["AXISBANK.NS","BHARELEC.NS","GODFPHIL.NS","HDFCBANK.NS","KOTAMAHI.NS","LARSTOUB.NS","REDI.NS","RELINDUS.NS","TATASTEE.NS"]
indexes = ["ITC.NS","BHARHEAV.NS","MARUSUZU.NS","WIPR.NS","EICHMOTO.NS","SHRIFINA.NS","IDFCFIRS.NS","HCLTECH.NS","JSWSTEE.NS"]
selected_symbols = defaultdict(list, {k: [] for k in indexes})

for file in os.listdir(input_folder_path):
    file_path = os.path.join(input_folder_path, file)
    with open(file_path, 'r') as f:
        lines = f.readlines()
        source_total_lines += len(lines)

        for line in lines:
            parts = line.strip().split("||")
            if len(parts) < 6:
                source_malformed_lines.append(line.strip())
                continue

            symbol = parts[4].strip()
            if symbol in selected_symbols:
                selected_symbols[symbol].append(line.strip())
            
            source_symbol_tick_dict[symbol] = source_symbol_tick_dict.get(symbol, 0) + 1

print(f"Source Total Ticks: {source_total_lines}")
print(f"Malformed Lines in Source: {len(source_malformed_lines)}")
print(f"Valid Ticks Stored: {sum(source_symbol_tick_dict.values())}")

Source Total Ticks: 3705688
Malformed Lines in Source: 3699
Valid Ticks Stored: 3701989


In [5]:
import pandas as pd

output_folder_path = r"C:\Users\kaustubh.keny\Projects\OUTPUTS\ticker-ops\data\17-09-2025"

output_total_ticks = 0
output_symbol_tick_dict = {}
output_errors = []

for file in os.listdir(output_folder_path):
    file_path = os.path.join(output_folder_path, file)
    try:
        df = pd.read_csv(file_path)
        ticks = df.shape[0] #excludes the header 
        symbol = file.replace(".csv", "")
        output_symbol_tick_dict[symbol] = ticks
        output_total_ticks += ticks
    except Exception as e:
        output_errors.append((file, str(e)))

print(f"Output Total Ticks: {output_total_ticks}")
print(f"Output Files Processed: {len(output_symbol_tick_dict)}")
print(f"Files with Errors: {len(output_errors)}")

Output Total Ticks: 3701989
Output Files Processed: 17356
Files with Errors: 0


In [ ]:
#ticks per index
import pandas as pd
output_cleaned = {k.replace('.csv', ''): v for k, v in output_symbol_tick_dict.items()}

all_symbols = set(source_symbol_tick_dict.keys()) | set(output_cleaned.keys())
tick_report = []
for symbol in sorted(all_symbols):
    source_count = source_symbol_tick_dict.get(symbol, 0)
    output_count = output_cleaned.get(symbol, 0)
    missing = source_count - output_count
    tick_report.append((symbol, source_count, output_count, missing))

# Display report
tick_report = sorted(tick_report, key = lambda x: x[1], reverse=True)
print(f"{'Symbol':25} | {'Source':>6} | {'Output':>6} | {'Missing':>7}")
print("-" * 55)
for symbol, src, out, miss in tick_report:
    print(f"{symbol:25} | {src:6} | {out:6} | {miss:7}")


# df = pd.DataFrame(tick_report,columns=["Symbol","Source","Output","Missing"])
# df.to_excel("Symbol_Count.xlsx")


DATA VALIDATION SHEET

In [ ]:
import pandas as pd

priority_keys = ["2", "4", "6", "5", "7", "3"]
key_to_label = {
    "10": "DATETIME",
    "2": "Ltp",
    "4": "BuyPrice",   # bid
    "6": "BuyQty",
    "5": "SellPrice",  # ask
    "7": "SellQty",
    "3": "Ltq"
}

for name, values in selected_symbols.items():
    cleaned_rows = []

    for tick in values:
        tick_data = tick.split("||")[-1]
        kv_pairs = [kv for kv in tick_data.split("~") if "=" in kv]
        kv_dict = {kv.split("=")[0]: kv.split("=")[1] for kv in kv_pairs}
        timestamp = kv_dict.get("10", None)
        if timestamp and " " in timestamp:
            date_part, time_part = timestamp.split(" ")
        else:
            date_part, time_part = "Nil", "Nil"
        row = [date_part, time_part]
        for key in priority_keys:
            row.append(kv_dict.get(key, ""))

        remaining_keys = [k for k in kv_dict.keys() if k not in priority_keys and k != "10"]
        for key in remaining_keys:
            row.append(kv_dict[key])

        cleaned_rows.append(row)

    
    header = ["DATE", "TIME"]
    header += [key_to_label.get(k, f"KEY_{k}") for k in priority_keys]

   
    max_cols = max(len(row) for row in cleaned_rows)
    while len(header) < max_cols:
        extra_idx = len(header) - len(priority_keys) - 2 + 1  # offset for DATE, TIME
        header.append(f"EXTRA_COL_{extra_idx}")
        
    df = pd.DataFrame(cleaned_rows)
    df.to_excel(f"{name}_tick.xlsx", index=False, header=header)